# **CMPE -297**

NLP Assignment

Aditya Sahu 

In this demo, BERT for question answering model will be explored from the huggingface library

In Part 1 of the demo, we will use a fine-tuned BERT on the **SQuAD** dataset and apply it (test) it) on the **CoQA** dataset.  In Part 2 of the demo you will learn how to fine tune BERT for question answering on the **SQuAD** dataset yourselves.

# PART 1

### Initialization & Setup


In [94]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
import pandas as pd
import json
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

### Loading the CoQA dataset

In [96]:
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...,...
7194,1,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,1,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,1,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,1,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


In [97]:
coqa["data"][0].keys()

dict_keys(['source', 'id', 'filename', 'story', 'questions', 'answers', 'name'])

### Inspecting the data

In [98]:
coqa["data"][0]

{'source': 'wikipedia',
 'id': '3zotghdk5ibi9cex97fepx7jetpso7',
 'filename': 'Vatican_Library.txt',
 'story': 'The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts, to 

The **CoQA** dataset contains ~7200 rows, and each row contains one paragraph and multiple question and answer pairs related to that paragraph.

If we print the first row, we see that there are 20 questions and answers for the first paragraph and that answers are in the form of start index and end index within the paragraph.  This is the standard format of any closed domain question answering dataset.

In [99]:
# deleting an unnecessary column
del coqa["version"]
coqa

,data
0,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."
...,...
7194,"{'source': 'gutenberg', 'id': '34j10vatjfyw0ao..."
7195,"{'source': 'cnn', 'id': '3vj40nv2qinjocrcy7k4z..."
7196,"{'source': 'race', 'id': '3rjsc4xj10uw0to3vq0v..."
7197,"{'source': 'wikipedia', 'id': '3gs6s824sqxty8v..."


### Converting the CoQA dataset to a more convenient format
We convert the CoQA dataset to a more convenient format by creating one question and answer pair per row.  This results in repeated content in the "text" column - once per questions and answer for the respective paragraph, we will be repeating the paragraph in the "text" column. 

In [100]:
cols = ["text","question","answer"]
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)
new_df = pd.DataFrame(comp_list, columns = cols) 
new_df

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project
...,...,...,...
108642,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was a sub?,Xabi Alonso
108643,(CNN) -- Cristiano Ronaldo provided the perfec...,Was it his first game this year?,Yes
108644,(CNN) -- Cristiano Ronaldo provided the perfec...,What position did the team reach?,third
108645,(CNN) -- Cristiano Ronaldo provided the perfec...,Who was ahead of them?,Barca.


### Loading BERT fine-tuned on SQuAD
Loading BERT for question answering which is already fine-tuned on SQuAD, as well as the corresponding BERT tokenizer (each pre-trained BERT model has a corresponding tokenizer)


In [101]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

### Experimenting with BERT


In [102]:
# picking out a random question and answer pair from the dataset
random_num = np.random.randint(0,len(new_df))
question = new_df["question"][random_num]
text = new_df["text"][random_num]
print("Question: ", question)
print("Text: ", text)

Question:  What did he say about the experience?
Text:  How much do you know about Albert Einstein? Albert Einstein, born on March 14, 1579 in Germany, was a great scientist in the world. He was strange because he hated haircuts and new clothes. He believed in peace. All his life, he hated war. However, his most famous idea, E=mc2, helped create the world's most dangerous weapon . Many people think he was the smartest person in the world. But Einstein said that _ What did he like? Einstein liked learning sailing . He sailed in small boats all his life. He once joked, "Sailing is the sport that takes the least energy!" When Einstein was a child, his mother made him take violin lessons. At first, he didn't like the violin. But then he learned to love music and became a good violinist. Later, he said, "Love is the best teacher." Why is the sky blue? In 1910, Einstein asked a question which many children often ask, "Why is the sky blue?" After his careful research, he answered the question

In [103]:
new_df["answer"][random_num]

'"Love is the best teacher."'

In [104]:
# tokeninzing the question and answer pair
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 272 tokens.


We inspect the resulting tokens and observe that each word is assigned a unique token, and that some rare words are getting split into multiple tokens. The token 101 is always the first token indicating the start of the input text, and token 102 is the separator token, which comes between the question and the answer and also at the end

In [105]:
# inspecting the resulting tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
what       2,054
did        2,106
he         2,002
say        2,360
about      2,055
the        1,996
experience   3,325
?          1,029
[SEP]        102
how        2,129
much       2,172
do         2,079
you        2,017
know       2,113
about      2,055
albert     4,789
einstein  15,313
?          1,029
albert     4,789
einstein  15,313
,          1,010
born       2,141
on         2,006
march      2,233
14         2,403
,          1,010
157       17,403
##9        2,683
in         1,999
germany    2,762
,          1,010
was        2,001
a          1,037
great      2,307
scientist   7,155
in         1,999
the        1,996
world      2,088
.          1,012
he         2,002
was        2,001
strange    4,326
because    2,138
he         2,002
hated      6,283
hair       2,606
##cut     12,690
##s        2,015
and        1,998
new        2,047
clothes    4,253
.          1,012
he         2,002
believed   3,373
in         1,999
peace      3,521
.          1,012
all        

In [106]:
# Visualizing the number of token in question and text
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
num_seg_a = sep_idx + 1
print("Number of tokens in segment A (question): ", num_seg_a)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B (answer): ", num_seg_b)

SEP token index:  9
Number of tokens in segment A (question):  10
Number of tokens in segment B (answer):  262


In [107]:
#creating the segment ids and making sure every input token has a segment id
segment_ids = [0]*num_seg_a + [1]*num_seg_b
assert len(segment_ids) == len(input_ids)

Now the tokens and the segment ids will be passed to the model

In [108]:
# token input_ids to represent the input and token segment_ids to differentiate 
# our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids = torch.tensor([segment_ids]))

Getting the start and end tokens from the output

In [109]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")

print("Text:\n{}".format(new_df["text"][random_num]))
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))

Text:
How much do you know about Albert Einstein? Albert Einstein, born on March 14, 1579 in Germany, was a great scientist in the world. He was strange because he hated haircuts and new clothes. He believed in peace. All his life, he hated war. However, his most famous idea, E=mc2, helped create the world's most dangerous weapon . Many people think he was the smartest person in the world. But Einstein said that _ What did he like? Einstein liked learning sailing . He sailed in small boats all his life. He once joked, "Sailing is the sport that takes the least energy!" When Einstein was a child, his mother made him take violin lessons. At first, he didn't like the violin. But then he learned to love music and became a good violinist. Later, he said, "Love is the best teacher." Why is the sky blue? In 1910, Einstein asked a question which many children often ask, "Why is the sky blue?" After his careful research, he answered the question like this: "It's because light is made up of many

Cleaning up the answer is needed when there are multiple tokens for a word. The double hash symbols indicate that a word split into multiple tokens (separated by ##)

In [110]:
# cleaning up the answer
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [111]:
print("Answer:\n{}.".format(answer.capitalize()))

Answer:
" love is the best teacher . ".


In [112]:
# retrieve and print the answer to this question that we had in the training set 
answer = new_df["answer"][random_num]
answer

'"Love is the best teacher."'

# PART 2

## Initialization & Setup

In [113]:
# importing required libraries
import requests
import json
import torch
import os
from tqdm import tqdm
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
from transformers import AdamW

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
# creating a directory in Google drive
if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'): os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

## Loading the SQuAD dataset

In [115]:
# getting the SQuAD dataset
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

File ‘train-v2.0.json’ already there; not retrieving.

File ‘dev-v2.0.json’ already there; not retrieving.



In [116]:
# Load the training dataset and inspecting it
with open('train-v2.0.json', 'rb') as f:
  squad = json.load(f)

In [117]:
# Each 'data' dict has two keys (title and paragraphs)
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

Here we see that for each topic there are multiple paragraphs, and for each paragraph there are mutliple question and answer pairs

In [118]:
# checking the number of topics
len(squad['data'])

442

In [119]:
print(squad['data'][0]['paragraphs'][0]['qas'][0].keys())
squad['data'][0]['paragraphs'][0]['context']

dict_keys(['question', 'id', 'answers', 'is_impossible'])


'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [120]:
# loading the data in triplets of context, questions and answers
def read_data(path):  
  
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

In [121]:
train_contexts, train_questions, train_answers = read_data('train-v2.0.json')
valid_contexts, valid_questions, valid_answers = read_data('dev-v2.0.json')

In [122]:
train_questions[0:3]

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?"]

In [123]:
print(f'There are {len(train_questions)} training set questions')
print(f'There are {len(valid_questions)} dev set questions')

There are 86821 training set questions
There are 20302 dev set questions


## Dataset pre-processing

In [124]:
# fixing some data quality issues
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers[:1000], train_contexts[:1000])
add_end_idx(valid_answers[:100], valid_contexts[:100])

## Fine-tuning BERT on SQuAD

In [125]:
# getting the model and its tokenizer (currently training on only 1000 rows as it is very time consuming)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_contexts[:1000], train_questions[:1000], truncation=True, padding=True)
valid_encodings = tokenizer(valid_contexts[:100], valid_questions[:100], truncation=True, padding=True)

In [126]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

Visualing the output of tokenizer, input ids are the token indices with padding of 0s, token_type_ids are different integers for different sequences and attention mask states which positions to give attention to while training

In [127]:
train_encodings["input_ids"][0]

[101,
 20773,
 21025,
 19358,
 22815,
 1011,
 5708,
 1006,
 1013,
 12170,
 23432,
 29715,
 3501,
 29678,
 12325,
 29685,
 1013,
 10506,
 1011,
 10930,
 2078,
 1011,
 2360,
 1007,
 1006,
 2141,
 2244,
 1018,
 1010,
 3261,
 1007,
 2003,
 2019,
 2137,
 3220,
 1010,
 6009,
 1010,
 2501,
 3135,
 1998,
 3883,
 1012,
 2141,
 1998,
 2992,
 1999,
 5395,
 1010,
 3146,
 1010,
 2016,
 2864,
 1999,
 2536,
 4823,
 1998,
 5613,
 6479,
 2004,
 1037,
 2775,
 1010,
 1998,
 3123,
 2000,
 4476,
 1999,
 1996,
 2397,
 4134,
 2004,
 2599,
 3220,
 1997,
 1054,
 1004,
 1038,
 2611,
 1011,
 2177,
 10461,
 1005,
 1055,
 2775,
 1012,
 3266,
 2011,
 2014,
 2269,
 1010,
 25436,
 22815,
 1010,
 1996,
 2177,
 2150,
 2028,
 1997,
 1996,
 2088,
 1005,
 1055,
 2190,
 1011,
 4855,
 2611,
 2967,
 1997,
 2035,
 2051,
 1012,
 2037,
 14221,
 2387,
 1996,
 2713,
 1997,
 20773,
 1005,
 1055,
 2834,
 2201,
 1010,
 20754,
 1999,
 2293,
 1006,
 2494,
 1007,
 1010,
 2029,
 2511,
 2014,
 2004,
 1037,
 3948,
 3063,
 4969,
 1010,
 36

In [128]:
train_encodings["token_type_ids"][0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [129]:
train_encodings["attention_mask"][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [130]:
# printing the number of training data samples
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 1000 context-question pairs


In [131]:
# adding the answers in the training set for fine tuning
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers[:1000])
add_token_positions(valid_encodings, valid_answers[:100])

In [132]:
# creating the dataset in the format it is required for fine tuning BERT
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [133]:
train_dataset = SQuAD_Dataset(train_encodings)
valid_dataset = SQuAD_Dataset(valid_encodings)

In [134]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

In [135]:
# loading the BERT model which we will fine tune
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [136]:
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [137]:
# Fine tuning it per batch
N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 5: 100%|██████████| 125/125 [01:32<00:00,  1.36it/s, loss=0.327]


In [138]:
# checking the performance
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)

100%|██████████| 13/13 [00:02<00:00,  4.87it/s]


In [139]:
acc

0.5721153846153846

# Homework assignment

Exercise 1: Fine-tune BERT for question answering on the CoQA dataset using the same process as shown in Part 2 for the SQuAD dataset.

Earlier we have downloaded Coqa train set converted it into dataframe named new_df, so we will just make a copy of it and use it.

For CoQa dev dataset, we just need its key and values from text, question and answer.

In [140]:
!wget -nc https://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json
!wget -nc https://downloads.cs.stanford.edu/nlp/data/coqa/coqa-dev-v1.0.json
with open('coqa-train-v1.0.json', 'rb') as f:
  coqa_train = json.load(f)
with open('coqa-dev-v1.0.json', 'rb') as f:
  coqa_test = json.load(f)

File ‘coqa-train-v1.0.json’ already there; not retrieving.

File ‘coqa-dev-v1.0.json’ already there; not retrieving.



In [141]:
# Each 'data' dict has seven keys 
coqa_test['data'][0].keys()

dict_keys(['source', 'id', 'filename', 'story', 'questions', 'answers', 'additional_answers', 'name'])

We observe that some of the answer are unknown in this CoQa Dataset, so we will remove those answers and questions related them.


In [142]:
coqa_train['data'][0]['answers'][13]

{'span_start': -1,
 'span_end': -1,
 'span_text': 'unknown',
 'input_text': 'unknown',
 'turn_id': 14,
 'bad_turn': 'true'}

In [143]:
# loading the data in triplets of context, questions and answers
def read_data_coqa(path):  
  with open(path, 'rb') as f:
    squad = json.load(f)
  skip_list=[]
  contexts = []
  questions = []
  answers = []
  c=-1
  count=-1
  for group in squad['data']:
    for i in group['answers']:
      c+=1
      if i['span_start']==-1:         # added condition to remove -1 values
        skip_list.append(c)
        continue
      answers.append(i)
  for group in squad['data']:
    for i in group['questions']:
      count+=1
      if count not in skip_list:
        context = group['story']
        question=i['input_text']
        contexts.append(context)
        questions.append(question)

  s=skip_list
  return contexts, questions, answers

train_contexts_coqa,train_questions_coqa,train_answers_coqa=read_data_coqa('coqa-train-v1.0.json')
test_contexts_coqa,test_questions_coqa,test_answers_coqa=read_data_coqa('coqa-dev-v1.0.json')


In [144]:
print(len(train_contexts_coqa),len(train_questions_coqa),len(train_answers_coqa))

107286 107286 107286


In [145]:
print(f'There are {len(train_questions_coqa)} training set questions')
print(f'There are {len(test_questions_coqa)} dev set questions')

There are 107286 training set questions
There are 7918 dev set questions


In [146]:
# fixing some data quality issues
def add_end_idx_coqa(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['input_text']
    start_idx = answer['span_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['span_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      if start_idx>1:
        answer['span_start'] = start_idx - 1
        answer['span_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      if start_idx>2:
        answer['span_start'] = start_idx - 2
        answer['span_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx_coqa(train_answers_coqa[:1000], train_contexts_coqa[1000])
add_end_idx_coqa(test_answers_coqa[:100], test_contexts_coqa[:100])

In [147]:
# getting the model and its tokenizer (currently training on only 1000 rows as it is very time consuming)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings_coqa = tokenizer(train_contexts_coqa[:1000], train_questions_coqa[:1000], truncation=True, padding=True)
test_encodings_coqa = tokenizer(test_contexts_coqa[:100], test_questions_coqa[:100], truncation=True, padding=True)
# printing the number of training data samples
no_of_encodings = len(train_encodings_coqa['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 1000 context-question pairs


In [148]:
# adding the answers in the training set for fine tuning
def add_token_positions_coqa(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['span_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['span_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions_coqa(train_encodings_coqa, train_answers_coqa[:1000])
add_token_positions_coqa(test_encodings_coqa, test_answers_coqa[:100])

In [149]:
# creating the dataset in the format it is required for fine tuning BERT
class CoQa_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [150]:
train_dataset = CoQa_Dataset(train_encodings_coqa)
valid_dataset = CoQa_Dataset(test_encodings_coqa)
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)
# loading the BERT model which we will fine tune
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
# checking the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')
# Fine tuning it per batch
N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

Working on cuda


Epoch 5: 100%|██████████| 125/125 [01:41<00:00,  1.23it/s, loss=1.5]


In [151]:
# checking the performance
model.eval()

acc = []

for batch in tqdm(valid_loader):
  with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_true = batch['start_positions'].to(device)
    end_true = batch['end_positions'].to(device)
    
    outputs = model(input_ids, attention_mask=attention_mask)

    start_pred = torch.argmax(outputs['start_logits'], dim=1)
    end_pred = torch.argmax(outputs['end_logits'], dim=1)

    acc.append(((start_pred == start_true).sum()/len(start_pred)).item())
    acc.append(((end_pred == end_true).sum()/len(end_pred)).item())

acc = sum(acc)/len(acc)
acc

100%|██████████| 13/13 [00:03<00:00,  4.19it/s]


0.1346153846153846

Exercise 2: Import the BERT model fine-tuned for classification and test its performance on any text classification dataset such as the twitter dataset.

Using IMDB Dataset for sentiment analysis using BERT model

In [152]:
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [153]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import random
from transformers import pipeline
import evaluate
from evaluate import evaluator
from transformers import DataCollatorWithPadding
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
imdb = load_dataset("imdb")

  0%|          | 0/3 [00:00<?, ?it/s]

In [155]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", do_lower_case=True)
max_length = 512
def preprocess_fn(a):
    return tokenizer(a["text"], truncation=True)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
tokenized_imdb = imdb.map(preprocess_fn, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [156]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [163]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [158]:
imdb.keys()

dict_keys(['train', 'test', 'unsupervised'])

Lets just test the current trained model on random "unsupervised" set of IMDB dataset. For example 11th paragraph

In [159]:
imdb['unsupervised'][11]['text']

"A lot of people criticize this movie as just being too dumb, but I'm just saying, give this movie a chance. It may not be the best film ever made, but at least it has clever gags in it and also includes a very young John Candy. I saw clips of the movie on the internet and I was amazed on how they were funny, especially the scene when Candy unintentionally almost destroys the entire police station while trying to find a pencil or when he hilariously fails his shooting test in the gun range. Since this movie was barely released in the U.S., you may want to track down a copy of it before it disappears completely. Also look for Peter Cook, Mickey Rooney, and Dick Emery, in what was to be his last film role, in the cast, as well."

In [160]:
input_text = str(imdb['unsupervised'][11]['text'])

model_pred = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device = 0)
model_pred(input_text)

[{'label': 'POSITIVE', 'score': 0.5319817662239075}]

Exercise 3: Fine-tune the BERT model from Exercise 2 on the text classification dataset you used for testing (in Exercise 2) and evaluate its performance (on a test set from the dataset that you set aside prior to fine tuning the model)

Now we can test our above model on the IMDB test set

In [161]:
our_evaluator = evaluator("text-classification")
data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(1000))
final = our_evaluator.compute(model_or_pipeline=model,data=data,tokenizer=tokenizer,metric="accuracy",label_mapping=label2id)
final['accuracy']

0.487

We are getting preety decent accuracy